In [ ]:
import os
import sys
import json
import copy
sys.path.append("..")
from lib.tweets import readTweetsFromFolder

searchString = 'map.decarbnow.abteil.org'
extractUrl = searchString + '/'

In [ ]:
info = readTweetsFromFolder(os.path.join('tweets', searchString))
        
dict(info[0])

In [ ]:
info[1]['entities']

In [ ]:
bl = {}
for t in info:
    print('------------------')
    print(t['id_str'])
    print(t['in_reply_to_status_id'])
    print(t['full_text'])
    #print('---')
    hl = []
    if t['entities']['hashtags']:
        hl = [h['text'] for h in t['entities']['hashtags']]
    print('Hashtags: ', ' '.join(hl))
    
    ul = []
    if t['entities']['urls']:
        ul = [u['expanded_url'] for u in t['entities']['urls']]
    ul = [u.split(extractUrl, 1)[1] for u in ul if extractUrl in u]
    print('Urls: ', ' '.join(ul))
    if (len(ul) > 0):
        bl[t['id_str']] = {
            'created_at': t['created_at'],
            'tweet_id': t['id_str'],
            'user_id': t['user']['id_str'],
            'parent_tweet_id': t['in_reply_to_status_id_str'],
            'parent_user_id': t['in_reply_to_user_id_str'],
            'url': ul[0],
            'hashtags': hl,
            'children': []
        }
bl

In [ ]:
blc = copy.deepcopy(bl)

tree = []

for e in blc.values():
    if e['parent_tweet_id'] and e['parent_tweet_id'] in blc:
        print('APPENDING ' + e['tweet_id'] + ' TO ' + e['parent_tweet_id'])
        blc[e['parent_tweet_id']]['children'].append(e)
    else:
        print('APPENDING ' + e['tweet_id'] + ' TO ROOT')
        tree.append(e)

tree

In [ ]:
final = {};
for e in tree:
    if len(e['children']) > 0:
        story = e['tweet_id']
        
        final[e['tweet_id']] = {
            'url': e['url'],
            'hashtags': e['hashtags'],
            'story': story
        }
        
        while len(e['children']) > 0:
            e = e['children'][0]
            final[e['tweet_id']] = {
                'url': e['url'],
                'hashtags': e['hashtags'],
                'story': story
            }
            
    else:
        final[e['tweet_id']] = {
            'url': e['url'],
            'hashtags': e['hashtags'],
        }

In [ ]:
final

In [ ]:
with open('../data/first.json', 'w') as json_file:
    json.dump(final, json_file)